# 잠재요인 협업 필터링 - `SVD(Singular Value Decomposition)` 활용
- 경진 대회에서 우승한 알고리즘 (넷플릭스 영화 추천)

In [3]:
from google.colab import files
up = files.upload()

Saving tags.csv to tags (1).csv
Saving README.txt to README.txt
Saving ratings_noh.csv to ratings_noh.csv
Saving ratings.csv to ratings.csv
Saving movies.csv to movies.csv


In [5]:
import pandas as pd
import numpy as np 

In [9]:
ratings = pd.read_csv('ratings.csv')
print(ratings.shape)
ratings.head()

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
!pip install scikit-surprise

In [12]:
from surprise import SVD, Reader
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds('ratings_noh.csv', reader = reader )

In [14]:
# 전체 데이터를 학습 데이터로 사용 
trainset = data_folds.build_full_trainset()
type(trainset)

surprise.trainset.Trainset

In [15]:
dir(trainset)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_global_mean',
 '_inner2raw_id_items',
 '_inner2raw_id_users',
 '_raw2inner_id_items',
 '_raw2inner_id_users',
 'all_items',
 'all_ratings',
 'all_users',
 'build_anti_testset',
 'build_testset',
 'global_mean',
 'ir',
 'knows_item',
 'knows_user',
 'n_items',
 'n_ratings',
 'n_users',
 'rating_scale',
 'to_inner_iid',
 'to_inner_uid',
 'to_raw_iid',
 'to_raw_uid',
 'ur']

In [16]:
# 모델 생성 및 학습 
model = SVD(n_epochs=20, n_factors=50, random_state=2022 ) # n_epochs 반복횟수, 
model.fit(trainset)

- 사용자 Id: 9 
- 영화 Id: 42 (title: Dead Presidents (1995))

In [17]:
# 영화 정보
mdf = pd.read_csv('movies.csv')
mdf.head(40).tail(5)

,movieId,title,genres
35,39,Clueless (1995),Comedy|Romance
36,40,"Cry, the Beloved Country (1995)",Drama
37,41,Richard III (1995),Drama|War
38,42,Dead Presidents (1995),Action|Crime|Drama
39,43,Restoration (1995),Drama


In [21]:
# 사용자 9번이 영화 42번 보았는지 확인 
movieIds = ratings[ratings.userId == 9]['movieId'] # ratings 데이터에서 userId == 9인 데이터들 중 movieId 데이터
movieIds[movieIds == 42].count() # 확인 
# 결과) 0 -> 존재X

0

In [24]:
# 사용자 9번 - 영화 42번에 대한 예상 평점 
uid, mid = str(9), str(42)
pred = model.predict(uid, mid , verbose=True) # verbose=True) : 화면에 보여줌

user: 9          item: 42         r_ui = None   est = 3.25   {'was_impossible': False}


In [23]:
model.predict(uid, mid )

Prediction(uid='9', iid='42', r_ui=None, est=3.249924377339538, details={'was_impossible': False})

- 사용자 9번이 보지 않은 영화중에서 예상점수가 가장 높은 Top 10

In [27]:
seen_movies = ratings[ratings.userId == 9]['movieId'].tolist()
total_movies = mdf.movieId # mdf: 위에서 정의한 영화 정보 데이터
unseen_movies = [movie for movie in total_movies if movie not in seen_movies] # 전체 영화에서 본 영화를 제외한 데이터

len(seen_movies), len(unseen_movies)
# 본 영화: 46개 , 안본영화: 9696개

(46, 9696)

In [31]:
# 사용자 9번이 보지 않은 영화들에 대한 예측값
predictions =[model.predict(str(9), str(mid)) for mid in unseen_movies] 
# str(9): 사용자 9번
predictions[:5]

[Prediction(uid='9', iid='1', r_ui=None, est=3.702922347424712, details={'was_impossible': False}),
 Prediction(uid='9', iid='2', r_ui=None, est=3.2274451421980412, details={'was_impossible': False}),
 Prediction(uid='9', iid='3', r_ui=None, est=3.0342513115122123, details={'was_impossible': False}),
 Prediction(uid='9', iid='4', r_ui=None, est=2.661778597408914, details={'was_impossible': False}),
 Prediction(uid='9', iid='5', r_ui=None, est=2.689490348191407, details={'was_impossible': False})]

In [32]:
# 정렬 함수 정의
# est 기준 오름차순 정렬 
def sortkey_est(pred):
  return pred.est # est : predictions 데이터

In [34]:
predictions.sort(key = sortkey_est, reverse=True) # reverse=True -> 내림차순 정렬 
predictions[:5]

[Prediction(uid='9', iid='318', r_ui=None, est=4.070330794979969, details={'was_impossible': False}),
 Prediction(uid='9', iid='1217', r_ui=None, est=4.063731956995097, details={'was_impossible': False}),
 Prediction(uid='9', iid='1261', r_ui=None, est=4.051908410348554, details={'was_impossible': False}),
 Prediction(uid='9', iid='1204', r_ui=None, est=4.0227662213503805, details={'was_impossible': False}),
 Prediction(uid='9', iid='3275', r_ui=None, est=4.011500870494226, details={'was_impossible': False})]

In [40]:
top_movie_ids = [int(pred.iid) for pred in predictions[:10]] # 상위 10개
top_movie_ratindgs= [pred.est for pred in predictions[:10]]
top_movie_titles = [mdf[mdf.movieId == mid ]['title'] for mid in top_movie_ids]
#top_movie_titles = mdf[mdf.movieId.isin(top_movie_ids)]['title']  -> 순서 고려가 안되서 위 코드로 
# 'mdf'의 'movieId'가 'top_movie_ids'에 존재하는 'mdf'의 데이터 중 'title' 데이터

In [41]:
# 데이터 프레임 구조로 변환 
top_df = pd.DataFrame({
    '영화명': top_movie_titles,
    '예상평점': top_movie_ratindgs
})
top_df.index.name = 'mdf_index'
top_df

,영화명,예상평점
mdf_index,,
0,"277 Shawshank Redemption, The (1994) Name: ...",4.070331
1,"918 Ran (1985) Name: title, dtype: object",4.063732
2,960 Evil Dead II (Dead by Dawn) (1987) Name...,4.051908
3,"906 Lawrence of Arabia (1962) Name: title, ...",4.022766
4,"2462 Boondock Saints, The (2000) Name: titl...",4.011501
5,3622 Amelie (Fabuleux destin d'Amélie Poula...,3.999696
6,"9071 Spotlight (2015) Name: title, dtype: o...",3.985348
7,"1258 Boogie Nights (1997) Name: title, dtyp...",3.979885
8,"46 Usual Suspects, The (1995) Name: title, ...",3.978625
